<a href="https://colab.research.google.com/github/Sandeep-ML-DL-NLP/NLP-Projects/blob/main/Sarcasm_Detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
# Data URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection

In [6]:
import numpy as np 
import pandas as pd 
import re 
import gensim
import json
import math 
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import KeyedVectors
import keras 
from keras.models import Sequential,Model
from keras import layers
from keras.layers import Dense,Dropout,Conv1D,GlobalMaxPooling1D 
import h5py

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def parse_data(file):
 for l in open(file,'r'):
    yield json.loads(l)
data = list(parse_data('/content/Sarcasm_Headlines_Dataset_v2.json'))
df = pd.DataFrame(data)

In [12]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [13]:
df.pop('article_link')

0        https://www.theonion.com/thirtysomething-scien...
1        https://www.huffingtonpost.com/entry/donna-edw...
2        https://www.huffingtonpost.com/entry/eat-your-...
3        https://local.theonion.com/inclement-weather-p...
4        https://www.theonion.com/mother-comes-pretty-c...
                               ...                        
28614    https://www.theonion.com/jews-to-celebrate-ros...
28615    https://local.theonion.com/internal-affairs-in...
28616    https://www.huffingtonpost.com/entry/andrew-ah...
28617    https://www.theonion.com/mars-probe-destroyed-...
28618    https://www.theonion.com/dad-clarifies-this-no...
Name: article_link, Length: 28619, dtype: object

In [14]:
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [15]:
len(df)

28619

In [40]:
# Clean Corpus 
def text_clean(corpus):
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
            p1 = p1.lower()
            qs.append(p1)
        cleaned_corpus = pd.concat([cleaned_corpus,pd.Series(' '.join(qs))])
    return cleaned_corpus

In [42]:
# Remove Stopword
def stopword_removal(corpus):
  stop = set(stopwords.words('english'))
  corpus = [ [x for x in x.split() if x not in stop] for x in corpus]
  return corpus

In [43]:
# Lemmatization 
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [44]:
# Stemming 
def stem(corpus, stem_type = None):
    if stem_type == 'snowball':
        stemmer = SnowballStemmer(language = 'english')
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    else :
        stemmer = PorterStemmer()
        corpus = [[stemmer.stem(x) for x in x] for x in corpus]
    return corpus

In [45]:
def preprocess(corpus, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):

    if cleaning == True:
        corpus = text_clean(corpus)
    if remove_stopwords == True:
        corpus = stopword_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]
    if lemmatization == True:
        corpus = lemmatize(corpus)
    if stemming == True:
        corpus = stem(corpus, stem_type)
    corpus = [' '.join(x) for x in corpus]
    return corpus

In [46]:
headlines = preprocess(df['headline'], lemmatization = True, remove_stopwords = True)

<ipython-input-40-32ee2ede754a>:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cleaned_corpus = pd.Series()


In [79]:
headlines

['thirtysomething scientists unveil doomsday clock hair loss',
 'dem rep totally nail congress fall short gender racial equality',
 'eat veggies 9 deliciously different recipes',
 'inclement weather prevent liar get work',
 'mother come pretty close use word stream correctly',
 'white inheritance',
 '5 ways file tax less stress',
 'richard branson global warm donation nearly much cost fail balloon trip',
 'shadow government get large meet marriott conference room b',
 'lot parent know scenario',
 'lesbian consider father indiana amaze one',
 'amanda peet tell daughter sex special hug',
 'know regard current treatments ebola',
 'chris christie suggest hillary clinton blame boko haram kidnap hundreds schoolgirls',
 'ford develop new suv run purely gasoline',
 'uber ceo travis kalanick step trump economic advisory council',
 'area boy enter jump touch top doorways phase',
 'area man travel gurney',
 'leave person disabilities behind',
 'lin manuel miranda would like remind put phone away'

In [51]:
# Word2Vec
import gensim.downloader as api

# Load a pre-trained Word2Vec model
model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [52]:
MAX_LENGTH = 10
VECTOR_SIZE = 300

In [62]:
def vectorize_data(data):
    
    vectors = []
    
    padding_vector = [0.0] * VECTOR_SIZE
    
    for i, data_point in enumerate(data):
        data_point_vectors = []
        count = 0
        
        tokens = data_point.split()
        
        for token in tokens:
            if count >= MAX_LENGTH:
                break
            if token in model.key_to_index:
                data_point_vectors.append(model[token])
            count = count + 1
        
        if len(data_point_vectors) < MAX_LENGTH:
            to_fill = MAX_LENGTH - len(data_point_vectors)
            for _ in range(to_fill):
                data_point_vectors.append(padding_vector)
        
        vectors.append(data_point_vectors)
        
    return vectors

In [63]:
vectorized_headlines = vectorize_data(headlines)

In [64]:
for i, vec in enumerate(vectorized_headlines):
    if len(vec) != MAX_LENGTH:
        print(i)

In [65]:
len(vectorized_headlines[1])

10

In [66]:
len(vectorized_headlines)

28619

In [67]:
train_div = math.floor(0.7 * len(vectorized_headlines))
train_div

20033

In [68]:
X_train = vectorized_headlines[:train_div]
y_train = df['is_sarcastic'][:train_div]
X_test = vectorized_headlines[train_div:]
y_test = df['is_sarcastic'][train_div:]

print('The size of X_train is:', len(X_train), '\nThe size of y_train is:', len(y_train),
      '\nThe size of X_test is:', len(X_test), '\nThe size of y_test is:', len(y_test))

The size of X_train is: 20033 
The size of y_train is: 20033 
The size of X_test is: 8586 
The size of y_test is: 8586


In [70]:
X_train = np.reshape(X_train, (len(X_train), MAX_LENGTH, VECTOR_SIZE))
X_test = np.reshape(X_test, (len(X_test), MAX_LENGTH, VECTOR_SIZE))
y_train = np.array(y_train)
y_test = np.array(y_test)

In [72]:
FILTERS=8
KERNEL_SIZE=3
HIDDEN_LAYER_1_NODES=10
HIDDEN_LAYER_2_NODES=5
DROPOUT_PROB=0.35
NUM_EPOCHS=10
BATCH_SIZE=50

In [74]:
# Model building and training
model = Sequential()

model.add(Conv1D(FILTERS,
                 KERNEL_SIZE,
                 padding='same',
                 strides=1,
                 activation='relu', 
                 input_shape = (MAX_LENGTH, VECTOR_SIZE)))
model.add(GlobalMaxPooling1D())
model.add(Dense(HIDDEN_LAYER_1_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(HIDDEN_LAYER_2_NODES, activation='relu'))
model.add(Dropout(DROPOUT_PROB))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 10, 8)             7208      
                                                                 
 global_max_pooling1d_1 (Glo  (None, 8)                0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 10)                90        
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense_4 (Dense)             (None, 5)                 55        
                                                                 
 dropout_3 (Dropout)         (None, 5)                 0         
                                                      

In [75]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
training_history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)


Epoch 1/10
401/401 [==============================] - 5s 5ms/step - loss: 0.6520 - accuracy: 0.6195
Epoch 2/10
401/401 [==============================] - 2s 4ms/step - loss: 0.5637 - accuracy: 0.7206
Epoch 3/10
401/401 [==============================] - 1s 4ms/step - loss: 0.5211 - accuracy: 0.7604
Epoch 4/10
401/401 [==============================] - 1s 4ms/step - loss: 0.4898 - accuracy: 0.7810
Epoch 5/10
401/401 [==============================] - 1s 4ms/step - loss: 0.4636 - accuracy: 0.8011
Epoch 6/10
401/401 [==============================] - 1s 4ms/step - loss: 0.4413 - accuracy: 0.8163
Epoch 7/10
401/401 [==============================] - 3s 6ms/step - loss: 0.4251 - accuracy: 0.8256
Epoch 8/10
401/401 [==============================] - 4s 10ms/step - loss: 0.4064 - accuracy: 0.8380
Epoch 9/10
401/401 [==============================] - 3s 6ms/step - loss: 0.3932 - accuracy: 0.8441
Epoch 10/10
401/401 [==============================] - 2s 6ms/step - loss: 0.3777 - accuracy: 0.854

In [77]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7674
